### 1. Simulate a p-stack machine with a dynamical system over $\mathbb{Q}^{s+p}$
Without loss of generality, we let $p=2$ as all other machines are polynomially equivalent.

Given, $\mathcal{M} = (S, s_I, s_H, \theta_{0}, \theta_{1}, \theta_{2}, \ldots, \theta_{p})$
$$ \theta_{0} : S \times \{0,1\}^{2p} \rightarrow S $$
$$ \theta_{i} : S \times \{0,1\}^{2p} \rightarrow \{(1, 0, 0), (1/4,0,1/4), (1/4,0,3/4), (4,-2,-1)\} $$
for $i = 1,..,p$

Instantaneous description of  $\mathcal{M}$ is $\mathcal{X} := S \times \mathcal{C}^p$, where $\mathcal{C}$ is the "Cantor 4-set"

The complete dynamics map of $\mathcal{M}$:
$$\mathcal{P} : \mathcal{X} \rightarrow \mathcal{X}$$
is defined as:
$$\mathcal{P} := [ \theta_{0}(s, a_1,..,a_p, b_1,..,b_p),
\theta_{1}(s, a_1,..,a_p, b_1,..,b_p)\cdot(q_1, a_1, 1),
\ldots
\theta_{p}(s, a_1,..,a_p, b_1,..,b_p)\cdot(q_p, a_p, 1),
]$$


In [1]:
from collections import defaultdict as dd
from fractions import Fraction
import numpy as np

s, p = 6, 2
state = np.zeros((s))
stacks = np.zeros((p))

In [2]:
#Sample beta randomly
beta = np.random.rand(2**(2*p), s, s+1)
beta[beta > 0.7] = 1
beta[beta <= 0.7] = 0
beta = beta.astype(int)

#Sample gammas randomly
gamma = np.random.rand(2**(2*p), p, s+1)
gamma1 = np.zeros((2**(2*p), p, s+1), dtype=int)
gamma1[gamma < 0.1] = 1 #np.zeros((2**(2*p), p, s+1))

gamma2 = np.zeros((2**(2*p), p, s+1), dtype=int)
gamma2[gamma > 0.1] = 1
gamma2[gamma > 0.2] = 0

gamma3 = np.zeros((2**(2*p), p, s+1), dtype=int)
gamma3[gamma > 0.2] = 1
gamma2[gamma > 0.3] = 0

gamma4 = np.zeros((2**(2*p), p, s+1), dtype=int)
gamma4[gamma > 0.3] = 1
gamma4[gamma > 0.4] = 0

print(beta.shape, gamma1.shape, gamma4.shape, beta.sum(), gamma1.sum(), gamma4.sum())


(16, 6, 7) (16, 2, 7) (16, 2, 7) 198 25 29


In [3]:
#input = 10001
def delta(binarystring):
    result = Fraction(0)
    for i, a in enumerate(binarystring):
        if a == "0":
            result += 1 / (4 ** (i+1))
        else:
            result += 3 / (4 ** (i+1))
    return result

def sigma(x):
    if x <= 0: return 0
    if x >= 1: return 1
    return x

stacks[0] = delta("10001")

x_0 = 1 - np.sum(state)
state_with_x0 = np.empty((s+1))
state_with_x0[0] = x_0
state_with_x0[1:] = state


new_state = beta[2] @ state_with_x0
print(new_state)

new_stack = (gamma1[2] @ state_with_x0) * stacks
print(new_stack)

def top_and_empty(stacks):
    a = ""
    b = ""
    for q in stacks:
        if q <= 1/2: a+= "0"
        else: a += "1"

        if q == 0: b += "0"
        else: b += "1"

    return int(a+b, base=2)


new_state = state
while new_state[0] != 1:
    # get a1,a2,..,ap, b1,b2..,bp
    apbp = top_and_empty(stacks)
    x_0 = 1 - np.sum(new_state)
    state_with_x0 = np.empty((s+1))
    state_with_x0[0] = x_0
    state_with_x0[1:] = new_state

    new_state = beta[apbp] @ state_with_x0
    print(new_state)
    vsigma = np.vectorize(sigma)
    new_stack = (gamma1[apbp] @ state_with_x0) * stacks + (gamma2[apbp] @ state_with_x0) * (stacks/4 + 1/4) + (gamma3[apbp] @ state_with_x0) * (stacks/4 + 3/4) + (gamma4[apbp] @ state_with_x0) * (4*stacks - 2*(vsigma(4*stacks-2)) - 1)
    print(new_stack)

[1. 1. 1. 0. 0. 0.]
[0.83496094 0.        ]
[1. 0. 0. 1. 0. 1.]
[0.95874023 0.        ]



### 2. Simulate a $\mathcal{P}$ by a $\sigma$-processor net

Refer equation (10). Given $\beta_{ij}$, we want to compute the coefficients $c_r$ such that
$$\beta_{ij}(a_1,...,a_p, b_1,...,b_p)x = \sum_{r=1}^{2^{2p}} c_r \sigma(v_r \cdot \mu)$$
where $\mu = [1, a_1,...,a_p, b_1,...,b_p, x] \in \mathbb{Z}^{2p + 2}$ is the input

and $v_r \in \{0,1\}^{2p + 2}$ are projection vectors such that $v_r$ selects $r$ symbols from $[a_1, a_2,.., b_1, b_2,..]$ in the dot product $v_r \cdot \mu$. 

$x$ is always selected so the last bit is 1 for all $v_r$. The first bit is the negative of the sum of number of ones in v_r[1]..v_r[p]. This is because of for any $l_1, l_2, ..., l_k \in \{0,1\}$, the product
$$l_1l_2...l_k = \sigma(l_1 + l_2 + \cdots + l_k - k + 1)$$


E.g. for $p=2$, $v_0 = [0, 0, 0, 0, 0, 1]$ (no symbols selected), $v_1 = [-1, 1, 0, 0, 0, 1]$ and so on.

To find $c_r's$ we note that $c_r's$ are the coefficients in the polynomial representation of  $\beta_{ij}$. (See equation (12))
$$
\begin{equation}
\beta_{ij}(a_1,...,a_p, b_1,...,b_p)x = c_1 + c_2a_1 + \cdots + c_{2p+1}b_p
 + c_{2p+2}a_1a_2 \cdots + c_{2^{2p}}a_1a_2...b_1..b_p
\end{equation}
$$

Now, given the maps $\beta_{ij}$, we can set up $2^{2p}$ linear equations in $c_r$'s by substituing $a_1,...,a_p, b_1,...,b_p$ on the right hand side and the value $\beta_{ij}(a_1,...,a_p, b_1,...,b_p)$ on the left hand side.
$$Ax = b$$ 
where $x = [c_1, c_2,\ldots,c_{2^{2p}}] \in \mathbb{Z}^{2^{2p}}$, $b = [\beta_{ij}[00..00], \beta_{ij}[00..01],\ldots,\beta_{ij}[11..11]] \in \{0,1\}^{2^{2p}}$, and $A \in \{0,1\}^{(2^{2p})\times(2^{2p})}$ is the such that rows of $A$ represents substitutions of $a_1,...,a_p, b_1,...,b_p$ with $00..00, 00..01,\ldots,11..11$ respectively on the right hand side of (1).

There is some ingenuity required in computing $A$ efficiently (i.e. in constant time, instead of actually performing the substitutions) as shown in the code below.

We can now use $\verb|np.linalg.solve|$ to find $c_r$'s, for each of $\beta_{ij}$ for $i = \{1..s\}, j=\{0..s\}$.

In [5]:
# Given Beta_ij, compute c_ij_1 to c_ij_2^t by solving linear systems (t = 2p)
# Linear system: Ax = b
# How many such systems? ---> i*j for each i and j
# What are A, x and b?
# x = [c1, c2, .. c_2^t]
# b = [Beta_ij[0000], Beta_ij[0001], .. Beta_ij[1111]]
# A = lower triangular? No.
def isCoeffPresent(x, y):
    if ~(x | ~y) == 0: return 1
    return 0
    
A = np.fromfunction(lambda x,y: ~(x | ~y) == 0, (16,16), dtype=int)
A.astype(int)
b = [1,0,1,1,1,1,0,0,1,0,0,1,1,0,0,1]
c = np.linalg.solve(A, b)
c.astype(int)
print('Ac = b, c=', c)

# F3 neuron: for state x_i, there's 2^2p of them
# input: 1 a1,b1..ap,bp, x_i
# parameters: 16 v_i's of shape 2p + 2, last bit always 1.. [-sum_of_1s, 0,1,..1,1, 1]
# select none of di's, v0 = [0 0 0 0 0 1]
# select d1, v1 = [-1 1 0 0 0 1]
# select d1d2, v3 = [-2 1 1 0 0 1]
# fixed, mu = [1 d1 d2 d3 d4 x], neuron computes sigma(v dot mu)
def get_vi(i, n):
    v = np.zeros((n), dtype=int)
    j = 0
    while i != 0:
        d = i%2
        v[j] = d
        i = i//2
        j+=1
    v_ = np.empty(n+2, dtype=int)
    v_[1:n+1] = v
    v_[n+1] = 1
    v_[0] = -np.sum(v)
    return v_

Ac = b, c= [ 1. -1.  0.  1.  0.  1. -1. -1.  0.  0. -1.  1.  0. -1.  1.  1.]


In [6]:
## Compute vectors c for each i,j of beta_ij

A = np.fromfunction(lambda x,y: ~(x | ~y) == 0, (2**(2*p),2**(2*p)), dtype=int)
c = np.empty((s,s+1, 2**(2*p)), dtype=int)
print(c.shape)
for i in range(s):
    for j in range(s+1):
        b = [beta[x][i][j] for x in range(2**(2*p))]
        c[i][j] = np.linalg.solve(A, b)
print(c)

(6, 7, 16)
[[[ 1 -1  0  0 -1  1  0  0 -1  1  1 -1  1 -1 -1  1]
  [ 1 -1  0  0  0  0 -1  1 -1  2  0  0  0 -1  2 -2]
  [ 0  0  0  0  1 -1 -1  2  0  0  0  0 -1  1  1 -1]
  [ 0  0  0  0  0  0  1  0  0  0  1  0  1 -1 -3  1]
  [ 0  0  1 -1  1 -1 -2  2  0  1  0 -1 -1  0  1  1]
  [ 0  0  0  0  1  0  0 -1  0  1  0 -1  0 -2  0  2]
  [ 0  1  0 -1  1 -2  0  2  0  0  0  1  0  1 -1 -2]]

 [[ 1  0  0 -1 -1  0  0  2 -1  0  0  2  1  0  1 -4]
  [ 0  0  0  0  0  0  0  0  1  0 -1  1 -1  0  1 -1]
  [ 0  1  0 -1  1 -2 -1  2  1 -2 -1  3 -2  3  3 -5]
  [ 0  0  0  0  0  0  0  0  0  0  0  1  0  0  1 -2]
  [ 0  0  0  1  1  0 -1 -1  0  1  0 -2 -1 -1  2  2]
  [ 0  0  0  0  0  1  1 -2  0  0  1 -1  1 -2 -3  4]
  [ 0  0  1 -1  0  0 -1  1  0  0 -1  1  0  1  1 -2]]

 [[ 0  0  1  0  0  0 -1  1  0  0 -1  0  1  0  0 -1]
  [ 0  0  1 -1  1 -1 -1  2  1 -1 -1  2 -2  2  1 -3]
  [ 0  1  0  0  0 -1  0  0  0 -1  0  0  0  1  1 -1]
  [ 0  1  1 -2  1 -2 -2  3  0 -1 -1  2 -1  2  2 -3]
  [ 0  1  0 -1  0 -1  0  1  0  0  0  1  1 -1 -1  

We now have the vectors $c \in \mathbb{Z}^{2^{2p}}$ for each of $i, j$ for each of $\beta_{ij}$.

For the gamma functions $\gamma^1_{ij}, \gamma^2_{ij}, \gamma^3_{ij}, \gamma^4_{ij}$ representing the next stack actions (no action, push0, push1, pop, respectively), we can compute coeffecient vectors $c$ in exactly the same manner as above.

Currently we focus on computing the next states given the current state and contents of each stack.

$$ (x_1, \ldots, x_s, a_1,...,a_p, b_1,...,b_p) \rightarrow (x_1^{+}, \ldots, x_s^{+}) $$

F4 neurons contain $x_i$'s. $x_i^{+}$ is computed in two steps.
$$ x_i^{+} = \sum_{j=0}^s{\beta_{ij}(a_1,...,a_p, b_1,...,b_p)x_j} = \sum_{j=0}^s{\sum_{r=1}^{2^{2p}} c_r \sigma(v_r \cdot \mu)}$$ 

F3 neurons compute each of the terms in the inner summation i.e. $\sigma(v_r \cdot \mu)$. F2 neurons then compute the linear combination with $c_r$ and the outer summation, and hence the new states $x_i^{+}$

Let's look at the map from F4 to F3, restricting our attention to $x_0, \ldots, x_s, a_1,...,a_p, b_1,...,b_p$ in F4. F3 has for each $x_j$, $2^{2p}$ neurons. Hence, the shape of this map is $F_4: \mathbb{Q}^{((s+1)2^{2p})\times((s+1)+2p)}$.

The $(s+1) + 2p + 1$ columns of this map correspond to $x_0, \ldots, x_s, a_1,...,a_p, b_1,...,b_p$. The rows of this map are divided into $s+1$ groups of $2^{2p}$ rows in each group. The first group of rows selects $x_0$, the second group selects $x_1$ and so on and the $s+1$ th group selecting $x_s$ in the dot product with $\mu$. In each group, the last $2p+1$ elements are that of $v_r$, where the $+1$ th element is the $-k$ from above equation. $F_4$ also applies saturation $\sigma$ to the computed values.

Now, we look at the map $F_3: \mathbb{Q}^{(s+1)2^{2p}} \rightarrow \mathbb{Q}^{s}$ which computes the new states $x_i^{+}$ by performing linear combination with $c_r$'s. This map has the shape  $\mathbb{Q}^{s \times (s+1)2^{2p}}$. Each row $i$ is simply the coeffecient vectors $c_i$ stacked horizontally.

The code below implements these two maps.

In [11]:
def get_vr(r, n):
    v = np.zeros((n), dtype=int)
    j = 0
    while r != 0:
        d = r%2
        v[j] = d
        r = r//2
        j+=1
    v_ = np.empty(n+1, dtype=int)
    v_[:n] = v
    v_[n] = -np.sum(v)
    return v_

#input vector of dimension (s+1) + 2p
def F4(in_f4):
    #Augment in_f4 with constant 1
    in_f4 = np.hstack((in_f4, [1]))
    #Construct the F4 map as described above
    n_rows = (s+1) * (2**(2*p))
    n_cols = s+1 + 2*p + 1
    A = np.zeros((n_rows, n_cols), dtype=int)
    for i in range(n_rows):
        r = i % (2**(2*p)) #r is the index within row group
        group_n = i // (2**(2*p)) #the group number
        A[i][group_n] = 1
        A[i][s+1:] = get_vr(r, 2*p)
        vsigma = np.vectorize(sigma)
    return vsigma(A @ in_f4)

f4_vec = np.array([1, 0, 0, 0, 0, 0, 0, 0.25, 0, 1, 0], dtype=int)
f4_out = F4(f4_vec)

In [12]:
## F3 map
f4_out.shape

#input vector of dimension (s+1)*(2^2p)
def F3(in_f3):
    #the param matrix is simply c[][] but squeezed into 1 dimension
    A = np.reshape(c, (s, (s+1)*(2**(2*p))))
    return A @ in_f3
print(c.shape)
cc = np.reshape(c, (s, (s+1)*(2**(2*p))))
print(cc.shape)
F3(F4(f4_vec))

(6, 7, 16)
(6, 112)


array([0, 0, 0, 1, 0, 0])

In [13]:
## TO-DO, implement next-stack action maps F2 and F1.
## Then the code for complete RNN (Universal net) is:
## while state != halting_state
##     state = F1(F2(F3(F4(state))))
## return state[index_of_stack_1] #contains encoding of the output.s